# Orbite di Keplero

Kepler found that the orbital period $\tau$ of a planet depends on its mean distance $R$ from the sun according to $\tau=c R^{\alpha}$ for a simple rational number $\alpha$. 

**(a)** Perform a linear least-squares fit from the following table in order to determine the most likely simple rational value of $\alpha$.

In [ ]:
#Import cell
using Markdown     #Visualizzazione di markdown in display
using Plots        #Per grafici
using CSV          #Per estrarre dati
using DataFrames
using Printf       #Per le stringhe del grafico
using LaTeXStrings
#non devo importare LinearAlgebra perchè è già dentro il file linear_systems.jl
include("c:\\ALL\\Stefano\\Bicocca\\3terzo_anno\\lab_comp\\lab_computazionale1\\librerie\\linear_systems.jl")
function autosave(fig, filename)
    path = "C:\\ALL\\Stefano\\Bicocca\\3terzo_anno\\lab_comp\\lab_computazionale1\\relazione\\immagini"
    savefig(fig, joinpath(path, filename))
end

In [ ]:
#Unpacking data
data = CSV.read("keplero.csv", DataFrame)
name = data[!, 1]
distance = data[!, 2]
time = data[!, 3]
mass = data[!, 4]

In [ ]:
function keplerian_model(x, c, a)
    return c*x.^a
end
colors = ["#0F2080",  # blu scuro
    "#F5793A",  # arancione
    "#A95AA1",  # viola
    "#85C0F9"]  # azzurro

Linearizzo il modello $\tau = c R^{\alpha}$ con il logaritmo. \
I conti sono su carta, ma la morale è che uso come y il logaritmo del tempo, come matrice A una matrice m*2 in cui la prima colonna è fatta di 1 e la seconda dei logaritmi delle distanze. Come parametri incogniti ottengo $log(c)$ e $\alpha$. 
In generale mi aspetto che $\alpha = \frac{3}{2}$ e che la costante dipenda dal pianeta. Ma dato che nella formula della costante compare la massa del Sole come termine dominante, fa in modo che tale costante sia uguale per ogni pianeta.

In [ ]:
#Dato che il modello non è lineare nei parametri lo linearizzo con il logaritmo
n_meas = length(time)
b = log.(time)
A = zeros(n_meas, 2)
A[:,1] = ones(n_meas)
A[:,2] = log.(distance)

param = least_sq(A, b)
c_fit = exp(param[1])
a_fit = param[2]
println("c = ", c_fit)
println("α = ", a_fit)
println("α attesa = ", 3/2)

**(b)** Make a plot of the data and the result of the fit.

In [ ]:
m = 24         #minimo sulle x
M = 10000        #massimo sulle x
m_y = 10           #minimo sulle y
M_y = 100000           #massimo sulle y
#= ticks_x = [exp(i) for i in range(log(m), stop=log(M), length=9)]
ticks_y = [exp(i) for i in range(log(m_y), stop=log(M_y), length=9)] =#

ticks_x = distance
ticks_y = time

# Definisci funzioni formatter per arrotondare i numeri
format_tick(x) = @sprintf("%.0f", x)

fig = plot( xlabel = L"Distanza\ dal\ Sole\ [Mkm]",
           ylabel = L"Tempo\ di\ rivoluzione\ [d]",
           xscale=:log, yscale=:log,
           legend=:topleft,
           framestyle = :box,
           grid=true, gridalpha = 0.5,
           xticks = ticks_x, yticks = ticks_y,
           xformatter = format_tick, yformatter = format_tick
           )

scatter!(fig, 
        distance, 
        time, 
        label = L"Dati\ raccolti",
        color = colors[1],
        )

#Calcolo le liste dell'interpolazione:
x_fit = [i for i in range(m, stop=M, length=100)]
plot!(fig, 
      x_fit,
      keplerian_model(x_fit, c_fit, a_fit),
      label = latexstring("Interpolazione:\\; \\alpha = ", @sprintf("%.3f", a_fit)),
      color = colors[2],
      linewidth = 2,
      linestyle = :dash,
      )

autosave(fig, "2621.pdf")
display(fig)